# Deep Learning for Autonomous Vehicles

**Assignment: 	Traffic Light Recongition**

**Student Name: Nithin Kumar**

Here we attempt to classify traffic lights from the Bosch Small Traffic light dataset. We first import the training and test data (images and labels) and assemble into an array and dataframe, respectively. We also resize the images to speed up computation time. We then train our CNN with the training data and calculate the accuracy on the test data. We find we get around 60% accuracy on the test data and discuss this at the end of the file.

In [1]:
import yaml
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from matplotlib import patches
from sklearn import preprocessing
import keras
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.datasets import mnist
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
import time
from sklearn.preprocessing import OneHotEncoder
from keras.layers.convolutional import Convolution2D
from sklearn import metrics

def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

Using TensorFlow backend.


In [2]:
data_path = "C:/Users/nithi/Desktop/traffic/"

with open(data_path + "train.yaml", "r") as test_file:
    train_yaml = yaml.safe_load(test_file)
with open(data_path + "test.yaml", "r") as test_file:
    test_yaml = yaml.safe_load(test_file)
print(len(train_yaml))
print(len(test_yaml))

5093
8334


In [3]:
train_df = pd.DataFrame(columns=('path', 'x_min', 'y_min','x_max','y_max','target'))

data_path = "C:/Users/nithi/Desktop/traffic/"

# def image_scaling(img):
#     res = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
#     return res

print("creating train data")
x_train = []
# y_train = []
counter = 0
for datapoint in train_yaml:
    counter += 1
    if counter % 1000 == 0:
        print("Processed {} of {}".format(counter, len(train_yaml)))
    pathname = datapoint["path"]
    image = cv2.imread(data_path+pathname)
    resized = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
    if datapoint["boxes"]:
        for i in range(0,len(datapoint["boxes"])):
            xmin = datapoint["boxes"][i]['x_min']
            xmax = datapoint["boxes"][i]['x_max']
            ymin = datapoint["boxes"][i]['y_min']
            ymax = datapoint["boxes"][i]['y_max']
            
            if datapoint["boxes"][i]:
                if datapoint["boxes"][i]['label'].startswith("Green"):
                    classname = "Green"
                elif datapoint["boxes"][i]['label'].startswith("Red"):
                    classname = "Red"
                elif datapoint["boxes"][i]['label'].startswith("Yellow"):
                    classname = "Yellow"
                elif datapoint["boxes"][i]['label'].startswith("off"):
                    classname = "off"
                else:
                    print(datapoint["boxes"][i]['label'])
                    print("something wrong")
                dict1 = {'path': pathname, 'x_min': xmin, 'y_min': ymin, 'x_max': xmax, 'y_max': ymax, 'target': classname}
                train_df = train_df.append(dict1, ignore_index = True)
                x_train.append(resized)
print("Processed {} of {}".format(counter, len(train_yaml)))


creating train data
Processed 500 of 5093
Processed 1000 of 5093
Processed 1500 of 5093
Processed 2000 of 5093
Processed 2500 of 5093
Processed 3000 of 5093
Processed 3500 of 5093
Processed 4000 of 5093
Processed 4500 of 5093
Processed 5000 of 5093
Processed 5093 of 5093


array(['Green', 'Red', 'Yellow', 'off'], dtype=object)

In [4]:
print("creating test data")
test_df = pd.DataFrame(columns=('path', 'x_min', 'y_min','x_max','y_max','target'))

x_test = []
counter = 0
for datapoint in test_yaml:
    counter += 1
    if counter % 1000 == 0:
        print("Processed {} of {}".format(counter, len(test_yaml)))
    if datapoint["boxes"]:
        pathname = datapoint["path"]
        tmp = os.path.basename(pathname)
        pathname = data_path+'rgb/test/'+tmp
        image = cv2.imread(pathname)
        resized = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        for i in range(0,len(datapoint["boxes"])):
            xmin = datapoint["boxes"][i]['x_min']
            xmax = datapoint["boxes"][i]['x_max']
            ymin = datapoint["boxes"][i]['y_min']
            ymax = datapoint["boxes"][i]['y_max']
            
            if datapoint["boxes"][i]:
                if datapoint["boxes"][i]['label'].startswith("Green"):
                    classname = "Green"
                elif datapoint["boxes"][i]['label'].startswith("Red"):
                    classname = "Red"
                elif datapoint["boxes"][i]['label'].startswith("Yellow"):
                    classname = "Yellow"
                elif datapoint["boxes"][i]['label'].startswith("off"):
                    classname = "off"
                else:
                    print("something wrong")
                    
                dict1 = {'path': pathname, 'x_min': xmin, 'y_min': ymin, 'x_max': xmax, 'y_max': ymax, 'target': classname}
                test_df = test_df.append(dict1, ignore_index = True)
                x_test.append(resized)
print("Processed {} of {}".format(counter, len(test_yaml)))


creating test data
Processed 500 of 8334
Processed 1000 of 8334
Processed 1500 of 8334
Processed 2000 of 8334
Processed 2500 of 8334
Processed 3000 of 8334
Processed 3500 of 8334
Processed 4000 of 8334
Processed 4500 of 8334
Processed 5000 of 8334
Processed 5500 of 8334
Processed 6000 of 8334
Processed 6500 of 8334
Processed 7000 of 8334
Processed 7500 of 8334
Processed 8000 of 8334
Processed 8334 of 8334


In [5]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)
x_train_np = x_train_np.astype('float32')
x_test_np = x_test_np.astype('float32')
x_train_np /= 255
x_test_np /= 255
print(x_train_np.shape)
print(x_test_np.shape)

y_train_np = OneHotEncoder().fit_transform(train_df[['target']]).toarray()
y_test_np = OneHotEncoder().fit_transform(test_df[['target']]).toarray()

print(y_train_np.shape)
print(y_test_np.shape)

(10756, 224, 224, 3)
(13486, 224, 224, 3)
(10756, 4)
(13486, 4)


In [7]:
NUM_CHANNELS = 3
IMAGE_WIDTH = 224 
IMAGE_HEIGHT = 224
NUM_CLASSES = 4

model = Sequential([
  Convolution2D(16, 3, 3, border_mode='same', subsample=(2, 2), input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS), activation='relu'),
  MaxPooling2D(pool_size=(3, 3)),
  Dropout(0.2),

  Convolution2D(32, 3, 3, border_mode='same', activation='relu'),
  MaxPooling2D(pool_size=(3, 3)),
  Dropout(0.2),

  Convolution2D(64, 3, 3, border_mode='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.2),

  Flatten(),
  Dense(128, activation='relu'),
  Dropout(0.3),
  Dense(NUM_CLASSES, activation='softmax'),
])
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

C:\Users\nithi\anaconda3\envs\tfnew4\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), input_shape=(224, 224,..., activation="relu", strides=(2, 2), padding="same")`
  import sys
C:\Users\nithi\anaconda3\envs\tfnew4\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  if sys.path[0] == '':
C:\Users\nithi\anaconda3\envs\tfnew4\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  app.launch_new_instance()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 112, 112, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 37, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
______________________________________

In [8]:
batch_size = 128
epochs = 8

start_time = time.time()

model.fit(x_train_np, y_train_np,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2)
score = model.evaluate(x_test_np, y_test_np, verbose=0)

print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))


Epoch 1/8
 - 46s - loss: 1.0577 - accuracy: 0.4872
Epoch 2/8
 - 40s - loss: 1.0366 - accuracy: 0.4981
Epoch 3/8
 - 33s - loss: 1.0324 - accuracy: 0.5043
Epoch 4/8
 - 32s - loss: 1.0238 - accuracy: 0.5147
Epoch 5/8
 - 31s - loss: 1.0114 - accuracy: 0.5304
Epoch 6/8
 - 31s - loss: 1.0046 - accuracy: 0.5344
Epoch 7/8
 - 31s - loss: 0.9956 - accuracy: 0.5396
Epoch 8/8
 - 31s - loss: 0.9905 - accuracy: 0.5461
Test loss: 0.820100229951159
Test accuracy: 0.5998072028160095
Elapsed time: 0:05:10.66


In [9]:
y_test = np.argmax(y_test_np,axis=1)
model_pred = model.predict(x_test_np)
pred = np.argmax(model_pred,axis=1)


score = metrics.accuracy_score(y_test, pred)
print('Accuracy: {}'.format(score))

r = pd.DataFrame( { 'y': y_test, "pred": pred})
r.to_csv("output.csv")

Accuracy: 0.599807207474418


We get around 60% accuracy on the test data. One issue I found that were not many training images with yellow or green lights. Most of them were red or off (driving). Our predictions on the test data mostly predicted 0 or 1 as shown in the output.csv file (corresponding to off or red).
The model itself is fairly simple - there are about 320,000 parameters with ReLU activation but we are not employing some of the state-of-the-art techniques such as Faster R-CNN. 
Furthermore, I had to downsample the images (to 224x224x3) or else the computation time was too long. 